In [29]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import subprocess
import time
from datetime import datetime
import subprocess


In [30]:
def load_mask_info():
    mask_info = pd.read_csv('/home/ubuntu/drive/EDA/merged_growth_real_real_final_dashboard.csv')
    return mask_info

mask_info = load_mask_info()

In [34]:
mask_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127857 entries, 0 to 127856
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   check_datetime       127857 non-null  object 
 1   측정일자                 127857 non-null  object 
 2   farm_code            127857 non-null  object 
 3   측정년도                 127857 non-null  int64  
 4   측정월                  127857 non-null  int64  
 5   input_image_name     127857 non-null  object 
 6   input_image_path     127857 non-null  object 
 7   img_id               127857 non-null  object 
 8   area_code            127857 non-null  object 
 9   shot_datetime        127857 non-null  object 
 10  before_img_name      127857 non-null  object 
 11  before_img_id        127857 non-null  object 
 12  mask_num             127857 non-null  object 
 13  mask_id              127857 non-null  object 
 14  size                 127857 non-null  float64
 15  pred_growth      

In [72]:
# selected_farm = st.selectbox('농가', mask_info['farm_code'].unique())
selected_farm = 'F0016'
selected_farm_data = mask_info[mask_info['farm_code'] == selected_farm]     

#농가별로 shot_datetime, mask_id, pred_growth 테이블로 정의해 재저장
mask_info_pivot_table = selected_farm_data.groupby(['shot_datetime','측정일자', 'mask_id'])['pred_growth'].first().reset_index()
mask_info_pivot_table['shot_datetime'] = pd.to_datetime(mask_info_pivot_table['shot_datetime'])

def period(start_date, end_date):
    start_date = datetime.strptime(start_date,'%Y-%m-%d %H:%M:%S')
    end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    return (start_date, end_date)

if selected_farm == 'F0016':
# 작기 기간 설정
    period_21_22 = period('2021-09-10 07:59:03', '2022-04-30 16:30:00')
    period_22_23 = period('2022-08-25 16:44:07', '2023-04-30 16:32:53')
    period_23_24 = period('2023-08-29 08:40:22', '2024-03-27 00:00:00')
else:
    period_21_22 = period('2021-09-10 07:59:03', '2022-05-01 00:00:00')
    period_22_23 = period('2022-08-25 16:44:07', '2022-05-01 00:00:00')
    period_23_24 = period('2023-08-29 08:40:22', '2024-03-27 00:00:00')    

# 작기 기간에 따라 데이터 필터링
mask_info_period = mask_info_pivot_table[(mask_info_pivot_table['shot_datetime'] >= period_23_24[0]) & (mask_info_pivot_table['shot_datetime'] <= period_23_24[1])]

#측정 날짜 기준으로 내림차순 정렬
mask_info_period = mask_info_period.sort_values(by='shot_datetime', ascending=False)

mask_cal = selected_farm_data.groupby(['unripe_to_semi-ripe','semi-ripe_to_ripe'])['mask_id'].first().reset_index()

# # 빈 리스트 생성
# mask_info_first_occurrence_list = []

# # 각 마스크 아이디 별로 필터링하여 조건에 맞는 데이터 추출
# for mask_id, group in mask_info_period.groupby('mask_id'):
#     unripe = group[group['pred_growth'] == 'unripe']['측정일자'].min()
#     semi_ripe = group[group['pred_growth'] == 'semi-ripe']['측정일자'].min()
#     ripe = group[group['pred_growth'] == 'ripe']['측정일자'].min()
#     # 딕셔너리를 리스트에 추가
#     mask_info_first_occurrence_list.append({'mask_id': mask_id, 'unripe_check_datetime': unripe, 'semi_ripe_check_datetime': semi_ripe, 'ripe_check_datetime': ripe})

# # 리스트를 데이터프레임으로 변환
# mask_info_first_occurrence = pd.DataFrame(mask_info_first_occurrence_list)

In [40]:
mask_cal

,unripe_to_semi-ripe,semi-ripe_to_ripe,mask_id
0,1.0,1.0,78f6d5d1c312e54ba2264fcf148e2bcd_1
1,1.0,2.0,834c51c59679c1fa70b6dd7344cc0cdd_1
2,1.0,3.0,cd8b2870b6edb9d699080cb2720a095f_1
3,1.0,4.0,d1a56c576f60253d648b35e6c407f087_1
4,1.0,5.0,c7523a61c1235d029671cf6f7cd603dc_3_0
...,...,...,...
83,15.0,8.0,00b3079ef5d5439393cad740b4492a8c_1
84,16.0,2.0,971d06c85695c906eadb8f0fdac6d1ba_0
85,16.0,4.0,3f6f815e14655e80d521791aa2086401_0
86,17.0,6.0,3563d2b4834fd5e32e8e604fd7efa981_8


In [44]:
import pandas as pd

# mask_cal 데이터프레임에서 unripe_to_semi-ripe 칼럼의 통계량 계산
min_value = mask_cal['unripe_to_semi-ripe'].min()
max_value = mask_cal['unripe_to_semi-ripe'].max()
mode_value = mask_cal['unripe_to_semi-ripe'].mode()  # 최빈값은 mode() 함수로 계산됩니다. 여러 최빈값이 있는 경우 첫 번째 값을 선택합니다.
mean_value = mask_cal['unripe_to_semi-ripe'].mean()

print("최소값:", min_value)
print("최대값:", max_value)
print("최빈값:", mode_value)
print("평균값:", mean_value)


최소값: 1.0
최대값: 20.0
최빈값: 0    2.0
Name: unripe_to_semi-ripe, dtype: float64
평균값: 5.8977272727272725


In [46]:
mask_cal['unripe_to_semi-ripe'].value_counts()

unripe_to_semi-ripe
2.0     11
1.0     10
4.0     10
5.0     10
3.0      9
6.0      8
7.0      6
9.0      5
8.0      4
11.0     3
15.0     2
16.0     2
10.0     2
12.0     2
14.0     1
13.0     1
17.0     1
20.0     1
Name: count, dtype: int64

In [47]:
mask_cal['semi-ripe_to_ripe'].value_counts()

semi-ripe_to_ripe
2.0     13
3.0     12
1.0     11
4.0     11
5.0     10
8.0      8
6.0      6
7.0      5
10.0     4
9.0      3
11.0     2
12.0     1
18.0     1
19.0     1
Name: count, dtype: int64

In [45]:
import pandas as pd

# mask_cal 데이터프레임에서 unripe_to_semi-ripe 칼럼의 통계량 계산
min_value = mask_cal['semi-ripe_to_ripe'].min()
max_value = mask_cal['semi-ripe_to_ripe'].max()
mode_value = mask_cal['semi-ripe_to_ripe'].mode()  # 최빈값은 mode() 함수로 계산됩니다. 여러 최빈값이 있는 경우 첫 번째 값을 선택합니다.
mean_value = mask_cal['semi-ripe_to_ripe'].mean()

print("최소값:", min_value)
print("최대값:", max_value)
print("최빈값:", mode_value)
print("평균값:", mean_value)

최소값: 1.0
최대값: 19.0
최빈값: 0    2.0
Name: semi-ripe_to_ripe, dtype: float64
평균값: 5.0


In [80]:
count_ripe = mask_info_pivot_table[mask_info_pivot_table['shot_datetime'] == '2024-03-30 16:42:52']['pred_growth'].value_counts()
count_ripe

pred_growth
semi-ripe    32
unripe        2
ripe          1
Name: count, dtype: int64

In [78]:
count_ripe = mask_info_pivot_table[mask_info_pivot_table['shot_datetime'] == '2024-03-29 16:42:55']['pred_growth'].value_counts()
count_ripe

pred_growth
semi-ripe    27
unripe        4
Name: count, dtype: int64

In [81]:
count_ripe = mask_info_pivot_table[mask_info_pivot_table['측정일자'] == '2024-03-31'].value_counts()
count_ripe

shot_datetime        측정일자        mask_id                              pred_growth
2024-03-31 08:42:46  2024-03-31  140c4230d21582d0eb4b99ff80b38985_5   semi-ripe      1
                                 15f59f626971f4bec0784adb96b744c7_4   semi-ripe      1
2024-03-31 16:42:49  2024-03-31  17c8fae85a547d3f593b39da813b775c_11  semi-ripe      1
                                 15f59f626971f4bec0784adb96b744c7_4   semi-ripe      1
                                 140c4230d21582d0eb4b99ff80b38985_5   semi-ripe      1
                                                                                    ..
2024-03-31 08:42:46  2024-03-31  c8fd94ced7bc149af40dfbe83c257bfc_16  semi-ripe      1
                                 ba68da91dea457857c64e5947b887b0e_14  semi-ripe      1
                                 b888f82df284a88b04384c16dae6c361_12  semi-ripe      1
                                 8f5c8adbb65e929e4ce9189d91e9889f_18  semi-ripe      1
2024-03-31 16:42:49  2024-03-31  f9a12e4845af539

In [82]:
count_ripe = mask_info_pivot_table[mask_info_pivot_table['shot_datetime'] == '2024-03-31 16:42:49']['pred_growth'].value_counts()
count_ripe

pred_growth
semi-ripe    24
ripe          2
Name: count, dtype: int64

In [22]:
# null이 없는 행만 필터링
mask_info_non_null = mask_info_first_occurrence.dropna()

# unripe_check_datetime가 null이 없는 행만 필터링
# unripe_check_datetime과 semi_ripe_check_datetime 모두 null이 아닌 행만 필터링
mask_info_unripe_semi_non_null = mask_info_first_occurrence.dropna(subset=['unripe_check_datetime', 'semi_ripe_check_datetime'])

# semi_ripe_check_datetime에서 unripe_check_datetime을 빼서 날짜 차이 계산
mask_info_unripe_semi_non_null['days_between'] = (mask_info_unripe_semi_non_null['semi_ripe_check_datetime'] - mask_info_unripe_semi_non_null['unripe_check_datetime']).dt.days

# semi_ripe_check_datetime가 null이 없는 행만 필터링
mask_info_ripe_semi_non_null = mask_info_first_occurrence.dropna(subset=['ripe_check_datetime', 'semi_ripe_check_datetime'])


In [23]:
mask_info_non_null

,mask_id,unripe_check_datetime,semi_ripe_check_datetime,ripe_check_datetime
6,03311a5c92cdcd7f791a5d21776ae93d_8,2024-01-08,2024-01-09,2024-01-12
37,067a480b75ed924d81d916362997bbfa_11,2024-01-06,2024-01-06,2024-01-07
65,0ddd00cfaa1217fc755d8016c3be51c6_5,2024-01-04,2024-01-04,2024-01-05
85,108b119c834536c97e2178faf87c2073_6,2024-01-06,2024-01-07,2024-01-09
93,11ba8dba9a678f4a2d4457b27df21979_6,2024-03-02,2024-03-02,2024-03-06
...,...,...,...,...
1326,f183357eeedf2e266c2c93d4fd8b0143_13,2023-12-17,2023-12-12,2023-12-15
1327,f183357eeedf2e266c2c93d4fd8b0143_14,2024-01-06,2024-01-02,2024-01-05
1346,f58deaa8833cd8cc03bb7d365211dccc_11,2024-03-22,2024-03-22,2024-03-25
1383,fbd5a41a368754cd076f200ccaacf644_15,2024-02-13,2024-02-14,2024-02-15


In [24]:
mask_info_unripe_semi_non_null

,mask_id,unripe_check_datetime,semi_ripe_check_datetime,ripe_check_datetime
6,03311a5c92cdcd7f791a5d21776ae93d_8,2024-01-08,2024-01-09,2024-01-12
9,034077223285cfc7af419ae0733e116c_12,2024-03-06,2024-03-06,NaN
10,034077223285cfc7af419ae0733e116c_13,2024-03-11,2024-03-11,NaN
13,034d9f3a1dfd88226d14254801f49a09_13,2024-01-12,2024-01-15,NaN
21,04d1c6295fc2901d807a757326767027_7,2024-01-15,2024-01-18,NaN
...,...,...,...,...
1383,fbd5a41a368754cd076f200ccaacf644_15,2024-02-13,2024-02-14,2024-02-15
1392,fc70dc0a6283f7d2d5778b3ea93e18b3_8,2024-03-14,2024-03-14,NaN
1394,fcfd5cec7695eb00e541fab8025bc02f_3,2024-01-10,2024-01-10,NaN
1397,fd1520f48e987168f98091f68d0c47f2_10,2024-03-23,2024-03-23,2024-03-26


In [25]:
mask_info_ripe_semi_non_null

,mask_id,unripe_check_datetime,semi_ripe_check_datetime,ripe_check_datetime
6,03311a5c92cdcd7f791a5d21776ae93d_8,2024-01-08,2024-01-09,2024-01-12
20,04a298305a37fd6aa05e84ab52765c07_4,NaN,2024-01-14,2024-01-15
33,066a1eef4d781f0b47e6f737f606c6ac_16,NaN,2023-12-28,2023-12-28
34,066a1eef4d781f0b47e6f737f606c6ac_17,NaN,2024-01-05,2024-01-06
37,067a480b75ed924d81d916362997bbfa_11,2024-01-06,2024-01-06,2024-01-07
...,...,...,...,...
1381,fbd5a41a368754cd076f200ccaacf644_13,NaN,2024-01-08,2024-01-07
1383,fbd5a41a368754cd076f200ccaacf644_15,2024-02-13,2024-02-14,2024-02-15
1390,fc0443333d4acabfe8f47ff8a73c21b0_9,NaN,2024-03-02,2024-03-03
1397,fd1520f48e987168f98091f68d0c47f2_10,2024-03-23,2024-03-23,2024-03-26


In [27]:
data = pd.read_csv('/home/ubuntu/drive/preprocessing/growth_preprocessed_filtering_data.csv')

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127857 entries, 0 to 127856
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   check_datetime       127857 non-null  object 
 1   측정일자                 127857 non-null  object 
 2   구역명                  127857 non-null  object 
 3   farm_code            127857 non-null  object 
 4   내부온도                 126222 non-null  float64
 5   내부습도                 126222 non-null  float64
 6   내부CO2                126222 non-null  float64
 7   근권온도                 126222 non-null  float64
 8   내부순간일사량              126222 non-null  float64
 9   측정년도                 127857 non-null  int64  
 10  측정월                  127857 non-null  int64  
 11  input_image_name     127857 non-null  object 
 12  input_image_path     127857 non-null  object 
 13  img_id               127857 non-null  object 
 14  area_code            127857 non-null  object 
 15  shot_datetime    

In [3]:

# selected_farm = st.selectbox('농가', mask_info['farm_code'].unique())
selected_farm_data = mask_info[mask_info['farm_code'] == 'F0016']   

#농가별로 shot_datetime, mask_id, pred_growth 테이블로 정의해 재저장
mask_info_pivot_table = selected_farm_data.groupby(['shot_datetime','측정일자', 'mask_id'])['pred_growth'].first().reset_index()
mask_info_pivot_table['shot_datetime'] = pd.to_datetime(mask_info_pivot_table['shot_datetime'])
selected_date = '2024-03-26'

# Convert selected_date to datetime format if it's not already
selected_date = pd.to_datetime(selected_date)

# Convert '측정일자' column to datetime format
mask_info_pivot_table['측정일자'] = pd.to_datetime(mask_info_pivot_table['측정일자'])
current_date = mask_info_pivot_table[mask_info_pivot_table['측정일자'] == selected_date]

current_date = current_date.sort_values(by='shot_datetime', ascending=False).iloc[0]['shot_datetime']
current_date = pd.to_datetime(current_date)

current_date_mask_info = mask_info_pivot_table[mask_info_pivot_table['shot_datetime'] == current_date]

mask_id = current_date_mask_info['mask_id'].unique()

filtered_data = pd.DataFrame()

semi_measurements = []
unripe_measurements = []

for mask_id in mask_id:
# mask_id와 pred_growth 조건을 만족하는 행을 필터링하여 임시 데이터프레임에 저장합니다.
    temp_data = selected_farm_data[(selected_farm_data['mask_id'] == mask_id) & (selected_farm_data['pred_growth'] != 'ripe')]

    # 'pred_growth'가 'semi_ripe'인 행들만 추출
    semi_ripe_data = temp_data[temp_data['pred_growth'] == 'semi-ripe']
    unripe_data = temp_data[temp_data['pred_growth'] == 'unripe']
    # print(semi_ripe_data)
    # print(unripe_data)
        
    # 'semi_ripe'인 행이 존재하는 경우
    if not semi_ripe_data.empty:
        # 측정일자가 가장 빠른 행을 선택하여 earliest_measurements 리스트에 추가
        semi_measurement = semi_ripe_data.loc[semi_ripe_data['check_datetime'].idxmin(), '측정일자']
        semi_measurement = pd.to_datetime(semi_measurement)
        semi_measurements.append(semi_measurement + timedelta(days=7))
    elif not unripe_data.empty:
        unripe_measurement = unripe_data.loc[unripe_data['check_datetime'].idxmin(), '측정일자']
        unripe_measurement = pd.to_datetime(unripe_measurement)
        unripe_measurements.append(unripe_measurement + timedelta(days=15))
    else:
        pass

# 'semi_measurements'와 'unripe_measurements' 두 리스트를 합침
all_measurements = semi_measurements + unripe_measurements

# 각 날짜별 개수를 세기
measurement_counts = Counter(all_measurements)
print(measurement_counts)

# 개수를 기준으로 내림차순 정렬하여 상위 3개를 출력
top_3_measurements = measurement_counts.most_common(3)

formatted_dates = [(top_3_measurements.strftime('%Y-%m-%d'), value) for top_3_measurements, value in top_3_measurements]

formatted_dates

Counter({Timestamp('2024-04-02 00:00:00'): 14, Timestamp('2024-04-01 00:00:00'): 6, Timestamp('2024-03-30 00:00:00'): 5, Timestamp('2024-03-31 00:00:00'): 4, Timestamp('2024-04-10 00:00:00'): 2, Timestamp('2024-04-04 00:00:00'): 1, Timestamp('2024-04-03 00:00:00'): 1})


In [4]:
top_3_measurements

[(Timestamp('2024-04-02 00:00:00'), 14),
 (Timestamp('2024-04-01 00:00:00'), 6),
 (Timestamp('2024-03-30 00:00:00'), 5)]

In [5]:

formatted_dates = [(top_3_measurements.strftime('%Y-%m-%d'), value) for top_3_measurements, value in top_3_measurements]

print(formatted_dates)


[('2024-04-02', 14), ('2024-04-01', 6), ('2024-03-30', 5)]


In [16]:
current_date = mask_info_pivot_table[mask_info_pivot_table['측정일자'] == selected_date]

current_date = current_date.sort_values(by='shot_datetime', ascending=False).iloc[0]['shot_datetime']
current_date = pd.to_datetime(current_date)

current_date_mask_info = mask_info_pivot_table[mask_info_pivot_table['shot_datetime'] == current_date]

mask_id = current_date_mask_info['mask_id'].unique()

ripe_measurements = []

for mask_id in mask_id:
# mask_id와 pred_growth 조건을 만족하는 행을 필터링하여 임시 데이터프레임에 저장합니다.
    temp_data = selected_farm_data[(selected_farm_data['mask_id'] == mask_id)]
    
    temp_data['check_datetime'] = pd.to_datetime(temp_data['check_datetime'])
    
    temp_data = temp_data[temp_data['check_datetime'] < current_date]

    # 'pred_growth'가 'semi_ripe'인 행들만 추출
    ripe_data = temp_data[temp_data['pred_growth'] == 'ripe']
    if not ripe_data.empty:
        ripe_measurement = ripe_data.loc[ripe_data['check_datetime'].idxmin(), '측정일자']
        ripe_measurement = pd.to_datetime(ripe_measurement)
        ripe_measurements.append(current_date - ripe_measurement)
        print('current_date:', current_date, 'ripe_measurement:', ripe_measurement)
    else:
        pass

current_date: 2024-03-26 16:42:48 ripe_measurement: 2024-03-25 00:00:00
current_date: 2024-03-26 16:42:48 ripe_measurement: 2024-03-25 00:00:00
current_date: 2024-03-26 16:42:48 ripe_measurement: 2024-03-26 00:00:00


/tmp/ipykernel_11548/1796402294.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['check_datetime'] = pd.to_datetime(temp_data['check_datetime'])
/tmp/ipykernel_11548/1796402294.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['check_datetime'] = pd.to_datetime(temp_data['check_datetime'])
/tmp/ipykernel_11548/1796402294.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [17]:
print(ripe_measurements)

[Timedelta('1 days 16:42:48'), Timedelta('1 days 16:42:48'), Timedelta('0 days 16:42:48')]


In [20]:
days_only = [str(delta).split(' ')[0] for delta in ripe_measurements]

# 각 일자별로 개수를 세기 위해 Counter를 사용하여 딕셔너리로 변환
count_by_day = dict(Counter(days_only))

print(count_by_day)

{'1': 2, '0': 1}
